### Miara odległości od normy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import plotly.express as px
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
import random
import plotly.graph_objs as go

In [ ]:
df_prep = pd.read_csv("..\chapter3\gasometry_prepared_data.csv")
cols=["BETET",	"CO2TET", "O2TET", "IONH"]
my_blue="#0064B2";
my_red="#D61600";

In [ ]:
df_dist=df_prep[["PACJENT_NR", "BADANIE_NR", "ZGON"]]

In [ ]:
df_miernik=df_prep.copy()
vars1=["BETET", "O2TET", 'HCO3ACTE', 'HCO3STTE',  'O2SATTET']
vars2=["IONH", "CO2TET"]

#wyliczenie wartości poza normą świadczące o zakwaszeniu organizmu
df_miernik[["BETET_kw", "O2TET_kw", 'HCO3ACTE_kw', 'HCO3STTE_kw',  'O2SATTET_kw']]=df_miernik[vars1].applymap(lambda x: 0 if x >= 0 else abs(x))
df_miernik[["IONH_kw", "CO2TET_kw"]]=df_miernik[vars2].applymap(lambda x: 0 if x <= 1 else x-1)

#całe odchylenia
df_miernik[["BETET_pn", "O2TET_pn", 'HCO3ACTE_pn', 'HCO3STTE_pn',  'O2SATTET_pn']] = df_miernik[vars1].applymap(lambda x: 0 if (x >= 0) and (x <= 1) else (abs(x) if x < 0 else x - 1))
df_miernik[["IONH_pn", "CO2TET_pn"]] = df_miernik[vars2].applymap(lambda x: 0 if (x >= 0) and (x <= 1) else (abs(x) if x < 0 else x - 1))

# Oblicz pierwiastek z sumy kwadratów elementów z każdego wiersza
df_dist['euclidean_kw'] = np.sqrt((df_miernik[["BETET_kw", "O2TET_kw", "IONH_kw", "CO2TET_kw"]]**2).sum(axis=1))
df_dist['euclidean_kw_skum']=df_dist.groupby('PACJENT_NR')['euclidean_kw'].cumsum()

df_dist['euclidean_all'] = np.sqrt((df_miernik[["BETET_pn", "O2TET_pn", "IONH_pn", "CO2TET_pn"]]**2).sum(axis=1))
df_dist['euclidean_all_skum']=df_dist.groupby('PACJENT_NR')['euclidean_all'].cumsum()

In [ ]:
# Podziel dane na dwie ramki danych
df_zgon_0 = df_dist[df_dist['ZGON'] == 0]
df_zgon_1 = df_dist[df_dist['ZGON'] == 1]

y='euclidean_all'

# fig0 = make_subplots(rows=1, cols=2, subplot_titles=['Zgon: nie', 'Zgon: tak'])
# Wykres dla ZGON=0
fig1 = px.line(df_zgon_0, x='BADANIE_NR', y=y, color='PACJENT_NR', 
              labels={'BADANIE_NR': 'BADANIE_NR', y: y, 'PACJENT_NR': 'Pacjent'}, width=800, height=800)
fig1.update_xaxes(title_text='Numer badania')
fig1.update_yaxes(title_text='Odległość od normy (Miernik 1)')
fig1.update_layout(showlegend=False)

# Wykres dla ZGON=1
fig2 = px.line(df_zgon_1, x='BADANIE_NR', y=y, color='PACJENT_NR', 
               labels={'BADANIE_NR': 'BADANIE_NR', y: y, 'PACJENT_NR': 'Pacjent'}, width=800, height=800)
fig2.update_xaxes(title_text='Numer badania')
fig2.update_yaxes(title_text='Odległość od normy (Miernik 1)')
fig2.update_layout(showlegend=False)

fig1.update_yaxes(range=[0, 15])
fig2.update_yaxes(range=[0, 15])


fig1.update_xaxes(range=[0, 65])
fig2.update_xaxes(range=[0, 65])

fig1.update_layout(
    xaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28),
    ),
    yaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28)
    )
)
fig2.update_layout(
    xaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28),
    ),
    yaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28)
    )
)
fig1.update_layout(template="plotly_white")
fig2.update_layout(template="plotly_white")

fig1.write_image("images4/"+"miernik1_pacjenci_wyleczeni"+ ".png", width=1000, height=800, scale=4, format="png")
fig2.write_image("images4/"+"miernik1_pacjenci_zmarli"+ ".png", width=1000, height=800, scale=4, format="png")
fig1.write_html("images4/miernik1_pacjenci_wyleczeni.html")
fig2.write_html("images4/miernik1_pacjenci_zmarli.html")

# Wyświetl oba wykresy
fig1.show()
fig2.show()

In [ ]:
# Podziel dane na dwie ramki danych
df_zgon_0 = df_dist[df_dist['ZGON'] == 0]
df_zgon_1 = df_dist[df_dist['ZGON'] == 1]

y='euclidean_all_skum'

# fig0 = make_subplots(rows=1, cols=2, subplot_titles=['Zgon: nie', 'Zgon: tak'])
# Wykres dla ZGON=0
fig1 = px.line(df_zgon_0, x='BADANIE_NR', y=y, color='PACJENT_NR', 
              labels={'BADANIE_NR': 'BADANIE_NR', y: y, 'PACJENT_NR': 'Pacjent'}, width=800, height=800)
fig1.update_xaxes(title_text='Numer badania')
fig1.update_yaxes(title_text='Skumulowana odległość od normy (Miernik 1)')
fig1.update_layout(showlegend=False)

# Wykres dla ZGON=1
fig2 = px.line(df_zgon_1, x='BADANIE_NR', y=y, color='PACJENT_NR', 
               labels={'BADANIE_NR': 'BADANIE_NR', y: y, 'PACJENT_NR': 'Pacjent'}, width=800, height=800)
fig2.update_xaxes(title_text='Numer badania')
fig2.update_yaxes(title_text='Skumulowana odległość od normy (Miernik 1)')
fig2.update_layout(showlegend=False)

# fig1.update_yaxes(range=[0, 15])
# fig2.update_yaxes(range=[0, 15])


fig1.update_yaxes(range=[0, 120])
fig2.update_yaxes(range=[0, 120])

fig1.update_xaxes(range=[0, 65])
fig2.update_xaxes(range=[0, 65])

fig1.update_layout(
    xaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28),
    ),
    yaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28)
    )
)
fig2.update_layout(
    xaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28),
    ),
    yaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28)
    )
)
fig1.update_layout(template="plotly_white")
fig2.update_layout(template="plotly_white")

fig1.write_image("images4/"+"miernik1_skum_pacjenci_wyleczeni"+ ".png", width=1000, height=800, scale=4, format="png")
fig2.write_image("images4/"+"miernik1_skum_pacjenci_zmarli"+ ".png", width=1000, height=800, scale=4, format="png")
fig1.write_html("images4/miernik1_skum_pacjenci_wyleczeni.html")
fig2.write_html("images4/miernik1_skum_pacjenci_zmarli.html")

# Wyświetl oba wykresy
fig1.show()
fig2.show()

In [ ]:
# Podziel dane na dwie ramki danych
df_zgon_0 = df_dist[df_dist['ZGON'] == 0]
df_zgon_1 = df_dist[df_dist['ZGON'] == 1]

y='euclidean_kw'

# fig0 = make_subplots(rows=1, cols=2, subplot_titles=['Zgon: nie', 'Zgon: tak'])
# Wykres dla ZGON=0
fig1 = px.line(df_zgon_0, x='BADANIE_NR', y=y, color='PACJENT_NR', 
              labels={'BADANIE_NR': 'BADANIE_NR', y: y, 'PACJENT_NR': 'Pacjent'}, width=800, height=800)
fig1.update_xaxes(title_text='Numer badania')
fig1.update_yaxes(title_text='Odległość od normy (Miernik 2)')
fig1.update_layout(showlegend=False)

# Wykres dla ZGON=1
fig2 = px.line(df_zgon_1, x='BADANIE_NR', y=y, color='PACJENT_NR', 
               labels={'BADANIE_NR': 'BADANIE_NR', y: y, 'PACJENT_NR': 'Pacjent'}, width=800, height=800)
fig2.update_xaxes(title_text='Numer badania')
fig2.update_yaxes(title_text='Odległość od normy (Miernik 2)')
fig2.update_layout(showlegend=False)

fig1.update_yaxes(range=[0, 15])
fig2.update_yaxes(range=[0, 15])


fig1.update_xaxes(range=[0, 65])
fig2.update_xaxes(range=[0, 65])

fig1.update_layout(
    xaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28),
    ),
    yaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28)
    )
)
fig2.update_layout(
    xaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28),
    ),
    yaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28)
    )
)
fig1.update_layout(template="plotly_white")
fig2.update_layout(template="plotly_white")

fig1.write_image("images4/"+"miernik2_pacjenci_wyleczeni"+ ".png", width=1000, height=800, scale=4, format="png")
fig2.write_image("images4/"+"miernik2_pacjenci_zmarli"+ ".png", width=1000, height=800, scale=4, format="png")
fig1.write_html("images4/miernik2_pacjenci_wyleczeni.html")
fig2.write_html("images4/miernik2_pacjenci_zmarli.html")

# Wyświetl oba wykresy
fig1.show()
fig2.show()

In [ ]:
# Podziel dane na dwie ramki danych
df_zgon_0 = df_dist[df_dist['ZGON'] == 0]
df_zgon_1 = df_dist[df_dist['ZGON'] == 1]

y='euclidean_kw_skum'

# fig0 = make_subplots(rows=1, cols=2, subplot_titles=['Zgon: nie', 'Zgon: tak'])
# Wykres dla ZGON=0
fig1 = px.line(df_zgon_0, x='BADANIE_NR', y=y, color='PACJENT_NR', 
              labels={'BADANIE_NR': 'BADANIE_NR', y: y, 'PACJENT_NR': 'Pacjent'}, width=800, height=800)
fig1.update_xaxes(title_text='Numer badania')
fig1.update_yaxes(title_text='Skumulowana odległość od normy (Miernik 2)')
fig1.update_layout(showlegend=False)

# Wykres dla ZGON=1
fig2 = px.line(df_zgon_1, x='BADANIE_NR', y=y, color='PACJENT_NR', 
               labels={'BADANIE_NR': 'BADANIE_NR', y: y, 'PACJENT_NR': 'Pacjent'}, width=800, height=800)
fig2.update_xaxes(title_text='Numer badania')
fig2.update_yaxes(title_text='Skumulowana odległość od normy (Miernik 2)')
fig2.update_layout(showlegend=False)

# fig1.update_yaxes(range=[0, 15])
# fig2.update_yaxes(range=[0, 15])


fig1.update_yaxes(range=[0, 120])
fig2.update_yaxes(range=[0, 120])

fig1.update_xaxes(range=[0, 65])
fig2.update_xaxes(range=[0, 65])

fig1.update_layout(
    xaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28),
    ),
    yaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28)
    )
)
fig2.update_layout(
    xaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28),
    ),
    yaxis=dict(
        tickfont=dict(size=18),
        title_font=dict(size=28)
    )
)
fig1.update_layout(template="plotly_white")
fig2.update_layout(template="plotly_white")

fig1.write_image("images4/"+"miernik2_skum_pacjenci_wyleczeni"+ ".png", width=1000, height=800, scale=4, format="png")
fig2.write_image("images4/"+"miernik2_skum_pacjenci_zmarli"+ ".png", width=1000, height=800, scale=4, format="png")
fig1.write_html("images4/miernik2_skum_pacjenci_wyleczeni.html")
fig2.write_html("images4/miernik2_skum_pacjenci_zmarli.html")

# Wyświetl oba wykresy
fig1.show()
fig2.show()

In [ ]:
df_pom=df_dist[["euclidean_kw", "PACJENT_NR", "ZGON", "BADANIE_NR"]]
len(df_pom[df_pom["euclidean_kw"]>8]["PACJENT_NR"].unique())
len(df_pom[(df_pom["euclidean_kw"] > 6) & (df_pom["ZGON"] == 1)]["PACJENT_NR"].unique())
len(df_pom[(df_pom["BADANIE_NR"] > 6)  & (df_pom["euclidean_kw"] > 4) & (df_pom["ZGON"] == 1)]["PACJENT_NR"].unique())


In [ ]:
df_6_measurements = df_dist.groupby('PACJENT_NR').head(6)

# Funkcja do obliczania współczynnika kierunkowego prostej
def calculate_slope(x, y):
    slope, intercept = np.polyfit(x, y, 1)
    return slope

# Oblicz współczynnik kierunkowy prostej dla każdego pacjenta
slopes = df_6_measurements.groupby('PACJENT_NR').apply(lambda group: calculate_slope(group['BADANIE_NR'], group['euclidean_kw_skum']))

unique_zgon = df_dist[['PACJENT_NR', 'ZGON']].drop_duplicates().set_index('PACJENT_NR')

# Połącz slopes z unique_zgon
slopes_df = slopes.to_frame(name='slope').join(unique_zgon)

slopes_df=slopes_df.reset_index(drop=True)
slopes_df

In [ ]:
plt.figure(figsize=(12, 6))

# Konwertowanie do numpy arrays
slope_zgon_0 = slopes_df[slopes_df['ZGON'] == 0]['slope'].to_numpy()
slope_zgon_1 = slopes_df[slopes_df['ZGON'] == 1]['slope'].to_numpy()

sns.histplot(slope_zgon_0, color='blue', label='ZGON = 0', bins=30)
sns.histplot(slope_zgon_1, color='red', label='ZGON = 1', bins=30)

plt.title('Rozkład wartości współczynnika kierunkowego dla ZGON = 0 i ZGON = 1')
plt.xlabel('Współczynnik kierunkowy')
plt.ylabel('Liczba próbek')
plt.legend()

In [ ]:
miernik2=df_dist[["euclidean_kw", "euclidean_kw_skum"]]
miernik2.shape

In [ ]:
data_to_extract=pd.concat([df_miernik, miernik2], axis=1)

In [ ]:
data_to_extract=data_to_extract.drop(columns="Unnamed: 0")

In [ ]:
data_to_extract.to_csv("data_to_extraction.csv")